# Heating Mesh Tally on CAD geometry made from Components

This constructs a reactor geometry from 3 Component objects each made from points.

The Component made include a breeder blanket, PF coil and a central column shield.

2D and 3D Meshes tally are then simulated to show nuclear heating, flux and tritium_production across the model.

This makes a 3D geometry and material for PF coil

In [ ]:
import paramak


pf_coil = paramak. PoloidalFieldCoil(
    height=50,
    width=50,
    center_point=(725, 25),
)

pf_coil.show()

This makes a 3D geometry and material for the centre column

In [ ]:
center_column = paramak.CenterColumnShieldHyperbola(
    height=1200,
    inner_radius=50,
    mid_radius=100,
    outer_radius=150,
)

center_column.show()

This makes a 3D geometry and material for breeder blanket. The azimuth_placement_angle argument is used to repeat the geometry around the Z axis at specified angles.

In [ ]:
blanket = paramak.BlanketConstantThicknessArcH(
    inner_mid_point=(500, 0),
    inner_upper_point=(400, 300),
    inner_lower_point=(400, -300),
    thickness= 100,
    rotation_angle=40,
    azimuth_placement_angle=[0, 45, 90, 135, 180, 225, 270, 315],
)

blanket.show()

This makes a reactor object from the three components

In [ ]:
my_reactor = paramak.Reactor([blanket, pf_coil,center_column])

my_reactor.show()

At this stage we can export the reactor geometry as stp files and make them avaialbe from download and viewing in FreeCAD.

In [ ]:
my_reactor.export_stp()

from IPython.display import FileLink
display(FileLink('blanket.stp'))
display(FileLink('pf_coil.stp'))
display(FileLink('center_column.stp'))
display(FileLink('Graveyard.stp'))

The next section defines the materials. This can be done using openmc.Materials or in this case strings that look up materials from the neutronics material maker.

In [ ]:
from neutronics_material_maker import Material

mat1 = Material.from_library(name='Li4SiO4')

mat2 = Material.from_library(name='copper')

mat3 = Material.from_library(name='WC')

This next step makes a simple point source. This uses native OpenMC commands but one could also use the openmc-plasma-source package https://github.com/fusion-energy/openmc-plasma-source/

In [ ]:
import openmc

# initialises a new source object
source = openmc.Source()

# sets the location of the source to x=0 y=0 z=0
source.space = openmc.stats.Point((100, 0, 0))

# sets the direction to isotropic
source.angle = openmc.stats.Isotropic()

# sets the energy distribution to 100% 14MeV neutrons
source.energy = openmc.stats.Discrete([14e6], [1])

This next section combines the geometry with the materials and specifies a few mesh tallies

In [ ]:
from dagmc_bounding_box import DagmcBoundingBox

# finds bounding box size from the geometry size
corners = DagmcBoundingBox('dagmc.h5m').corners()

tally1 = odw.MeshTally2D(
    mesh_resolution=(100, 100, 100),
    bounding_box= corners,
    tally_type="(n,Xa)",
    
)


import openmc
tallies = openmc.Tallies([tally1])

The next section produces download links for:

- vtk files that contain the 3D mesh results (open with Paraview)
- png images that show the resuls of the 2D mesh tally

In [ ]:
from openmc_mesh_tally_to_vtk import write_mesh_tally_to_vtk
# importing a package for converting regular mesh tallies to vtk files
# more details here https://github.com/fusion-energy/openmc_mesh_tally_to_vtk


# assumes you have a statepoint file from the OpenMC simulation
statepoint = openmc.StatePoint('statepoint.4.h5')

# this shows the tallies present in the statepoint file
print(statepoint.tallies)

# loads up a tally from the statepoint using it's name
my_tally = statepoint.get_tally(name='(n,Xa)_on_3D_mesh')

# converts the tally result into a VTK file
write_mesh_tally_to_vtk(
    tally=my_tally,
    filename = "vtk_file_from_openmc_mesh.vtk",
)

In [ ]:
from IPython.display import FileLink
display(FileLink('heating_on_3D_mesh.vtk'))
